In [ ]:
import requests

def get_moex_tickers():
    url = "https://iss.moex.com/iss/engines/stock/markets/shares/securities.json"
    params = {
        "iss.meta": "off"
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # проверка на ошибки
    data = response.json()
    
    # Данные о ценных бумагах находятся в блоке "securities"
    securities = data.get("securities", {})
    data_list = securities.get("data", [])
    columns = securities.get("columns", [])
    
    # Находим индекс колонки SECID, который содержит тикеры
    try:
        secid_index = columns.index("SECID")
    except ValueError:
        print("Поле SECID не найдено в ответе API!")
        return []
    
    tickers = [item[secid_index] for item in data_list]
    return tickers

if __name__ == '__main__':
    tickers = get_moex_tickers()
    print("Список тикеров:", tickers)

In [ ]:
len(tickers)

In [ ]:
import requests

def get_index_tickers(index_code):
    """
    Получает список тикеров, входящих в состав указанного индекса.
    
    :param index_code: Код индекса (например, 'IMOEX' для индекса Московской биржи)
    :return: Список тикеров (SECID) или пустой список, если данные не получены.
    """
    url = f"https://iss.moex.com/iss/compositions/{index_code}.json"
    params = {
        "iss.meta": "off"  # отключаем метаданные для упрощения ответа
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # выброс исключения при ошибке запроса
    except Exception as e:
        print(f"Ошибка запроса к MOEX ISS: {e}")
        return []
    
    data = response.json()
    components = data.get("components", {})
    data_list = components.get("data", [])
    columns = components.get("columns", [])
    
    # Ищем индекс колонки SECID, в которой содержатся тикеры
    try:
        secid_index = columns.index("SECID")
    except ValueError:
        print("Поле SECID не найдено в данных состава индекса!")
        return []
    
    tickers = [item[secid_index] for item in data_list]
    return tickers

if __name__ == "__main__":
    # Пример: получаем состав индекса IMOEX
    index_code = "IMOEX"  
    tickers = get_index_tickers(index_code)
    if tickers:
        print(f"Список тикеров для индекса {index_code}:")
        for ticker in tickers:
            print(ticker)
    else:
        print("Не удалось получить список тикеров.")

In [ ]:
import requests

def get_moex_index_tickers():
    url = "https://iss.moex.com/iss/engines/stock/markets/index/securities.json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        # Проверяем структуру ответа
        if "securities" in data and "data" in data["securities"]:
            tickers = [item[0] for item in data["securities"]["data"]]  # 'SECID' - первый элемент в списке
            return tickers
        else:
            print("Ошибка: Неверная структура данных")
            return None
    else:
        print(f"Ошибка при получении данных: {response.status_code}")
        return None

# Вывод списка тикеров
tickers = get_moex_index_tickers()
print(tickers)


In [ ]:
len(tickers)

In [ ]:
import requests

def get_moexog_tickers():
    url = "https://iss.moex.com/iss/index/MOEXOG/components.json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        try:
            tickers = [item[0] for item in data['components']['data']]  # Первый элемент массива — тикер
            return tickers
        except KeyError:
            print("Ошибка в структуре ответа:", data)
            return []
    else:
        print("Ошибка запроса:", response.status_code)
        return []

tickers = get_moexog_tickers()
print("Тикеры индекса MOEXOG:", tickers)

In [ ]:
import requests
from bs4 import BeautifulSoup

# Пример URL — уточните адрес нужной страницы с составом индекса
url = 'https://www.moex.com/ru/indicators/imoex'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Найдите нужные элементы. Например, если тикеры находятся в таблице:
table = soup.find('table', {'class': 'index-constituents'})
if table:
    rows = table.find_all('tr')
    for row in rows[1:]:  # пропускаем заголовок
        cols = row.find_all('td')
        # Предположим, первый столбец — тикер, второй — название компании
        if cols and len(cols) >= 2:
            ticker = cols[0].text.strip()
            name = cols[1].text.strip()
            print(f"{ticker}: {name}")
else:
    print("Не удалось найти таблицу с данными")

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL страницы
url = "https://smart-lab.ru/q/index_stocks/IMOEX/"

# Заголовки для запроса (имитация браузера)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Отправка GET-запроса
response = requests.get(url, headers=headers)

# Проверка успешности запроса
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Поиск таблицы с тикерами
    table = soup.find("table", {"class": "smartlab-table"})
    
    if table:
        rows = table.find_all("tr")[1:]  # Пропускаем заголовок таблицы
        tickers = []

        for row in rows:
            columns = row.find_all("td")
            if len(columns) > 1:
                ticker = columns[1].text.strip()  # Извлекаем название тикера
                tickers.append(ticker)

        print("Список тикеров:")
        print(tickers)
    else:
        print("Таблица не найдена на странице.")
else:
    print(f"Ошибка загрузки страницы: {response.status_code}")

In [ ]:
import requests

def get_moexbmi_tickers():
    # URL для получения состава индекса MOEXBMI в формате JSON
    url = "https://iss.moex.com/iss/compositions.json"
    params = {
        "index": "MOEXBMI"
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # проверка на успешный ответ
    except requests.RequestException as e:
        print("Ошибка при выполнении запроса:", e)
        return []

    data = response.json()
    compositions = data.get("compositions", {})
    columns = compositions.get("columns", [])
    rows = compositions.get("data", [])

    if not rows:
        print("Данные по тикерам не найдены.")
        return []

    # Обычно тикер содержится в столбце "SECID"
    try:
        secid_index = columns.index("SECID")
    except ValueError:
        print("Столбец SECID не найден в полученных данных.")
        return []

    # Извлекаем тикеры из строк
    tickers = [row[secid_index] for row in rows]
    return tickers

if __name__ == "__main__":
    tickers = get_moexbmi_tickers()
    if tickers:
        print("Список тикеров в составе индекса MOEXBMI:")
        for ticker in tickers:
            print(ticker)
    else:
        print("Не удалось получить список тикеров.")

In [ ]:
import requests

def get_moexbmi_tickers():
    # Новый URL для получения состава индекса через weightings
    url = "https://iss.moex.com/iss/index/weightings.json"
    params = {"index": "MOEXBMI"}
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # проверка на успешный ответ
    except requests.RequestException as e:
        print("Ошибка при выполнении запроса:", e)
        return []

    data = response.json()

    # В структуре ответа интересующий нас раздел называется "index/weightings"
    weightings = data.get("index/weightings", {})
    columns = weightings.get("columns", [])
    rows = weightings.get("data", [])

    if not rows:
        print("Данные по тикерам не найдены.")
        return []

    try:
        # Обычно тикер содержится в столбце "SECID"
        secid_index = columns.index("SECID")
    except ValueError:
        print("Столбец SECID не найден в полученных данных.")
        return []

    # Извлекаем тикеры из строк
    tickers = [row[secid_index] for row in rows]
    return tickers

if __name__ == "__main__":
    tickers = get_moexbmi_tickers()
    if tickers:
        print("Список тикеров, входящих в состав индекса MOEXBMI:")
        for ticker in tickers:
            print(ticker)
    else:
        print("Не удалось получить список тикеров.")

In [ ]:
pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_moexbmi_tickers():
    # URL страницы с составом индекса MOEXBMI на Smart‑Lab
    url = "https://smart-lab.ru/q/index_stocks/MOEXBMI/"
    
    # Задаём заголовок User-Agent, чтобы эмулировать браузер
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # проверка на ошибки HTTP
    except requests.RequestException as e:
        print("Ошибка при выполнении запроса:", e)
        return []
    
    # Парсим HTML-страницу
    soup = BeautifulSoup(response.text, "html.parser")
    
    # На сайте Smart‑Lab таблицы часто имеют класс "simple-little-table".
    # Попытаемся найти таблицу, в заголовке которой есть слово "Код" (это столбец с тикерами).
    tables = soup.find_all("table", class_="simple-little-table")
    target_table = None
    for table in tables:
        header = table.find("tr")
        if header and "Код" in header.get_text():
            target_table = table
            break

    if not target_table:
        print("Не удалось найти таблицу с составом индекса.")
        return []
    
    # Определяем индекс столбца, содержащего тикеры (в заголовке таблицы он называется "Код")
    header_row = target_table.find("tr")
    header_cells = header_row.find_all(["th", "td"])
    ticker_col_index = None
    for idx, cell in enumerate(header_cells):
        if cell.get_text(strip=True) == "Код":
            ticker_col_index = idx
            break

    if ticker_col_index is None:
        print("Не удалось определить колонку с тикерами.")
        return []
    
    # Извлекаем тикеры из строк таблицы (первая строка – заголовок, её пропускаем)
    tickers = []
    for row in target_table.find_all("tr")[1:]:
        cells = row.find_all("td")
        if len(cells) > ticker_col_index:
            ticker = cells[ticker_col_index].get_text(strip=True)
            tickers.append(ticker)
    
    return tickers

if __name__ == "__main__":
    tickers = get_moexbmi_tickers()
    if tickers:
        print("Список тикеров, входящих в состав индекса MOEXBMI:")
        for ticker in tickers:
            print(ticker)
    else:
        print("Не удалось получить список тикеров.")

In [ ]:
pip install investpy

In [ ]:
import investpy

def get_moexbmi_constituents():
    try:
        # Название индекса должно точно совпадать с тем, как оно представлено в investpy.
        # В данном случае используется 'MOEX BMI' для России.
        df = investpy.indices.get_index_constituents(index='IMOEX', country='russia')
        return df
    except Exception as e:
        print("Ошибка при получении данных с investpy:", e)
        return None

if __name__ == "__main__":
    constituents_df = get_moexbmi_constituents()
    if constituents_df is not None:
        print("Состав индекса MOEX BMI:")
        print(constituents_df)
    else:
        print("Не удалось получить состав индекса MOEX BMI.")

In [ ]:
investpy.indices.get_indices()

In [ ]:
import requests
import os
from time import sleep

def download_reports():
    companies = {
    "MSNG": "+МосЭнерго",
    "DATA": "iАренадата",
    "WUSH": "iВУШХолднг",
    "DIAS": "iДиасофт",
    "IVAT": "iИВА",
    "DELI": "iКаршеринг",
    "POSI": "iПозитив",
    "SOFL": "iСофтлайн",
    "MDMG": "MDMG-ао",
    "AKRN": "Акрон",
    "ALRS": "АЛРОСА ао",
    "APTK": "Аптеки36и6",
    "ASTR": "Астра ао",
    "AFLT": "Аэрофлот",
    "BANE": "Башнефт ап",
    "BSPB": "БСП ао",
    "VSEH": "ВИ.ру",
    "VSMO": "ВСМПО-АВСМ",
    "VTBR": "ВТБ ао",
    "GAZP": "ГАЗПРОМ ао",
    "GMKN": "ГМКНорНик",
    "FESH": "ДВМП ао",
    "DVEC": "ДЭК ао",
    "LEAS": "Европлан",
    "EUTR": "ЕвроТранс",
    "ZAYM": "Займер ао",
    "AQUA": "ИНАРКТИКА",
    "IRAO": "ИнтерРАОао",
    "LENT": "Лента ао",
    "LSRG": "ЛСР ао",
    "LKOH": "ЛУКОЙЛ",
    "MVID": "М.видео",
    "MGNT": "Магнит ао",
    "MGTS": "МГТС-4ап",
    "MTLR": "Мечел ао",
    "MTLRP": "Мечел ап",
    "CBOM": "МКБ ао",
    "VKCO": "МКПАО ВК",
    "GEMC": "МКПАО ЮМГ",
    "MAGN": "ММК",
    "MOEX": "МосБиржа",
    "MBNK": "МТС Банк",
    "MTSS": "МТС-ао",
    "NKNC": "НКНХ ап",
    "NKHP": "НКХП ао",
    "NLMK": "НЛМК ао",
    "NMTP": "НМТП ао",
    "BELU": "НоваБев ао",
    "NVTK": "Новатэк ао",
    "OGKB": "ОГК-2 ао",
    "OZPH": "ОзонФарм",
    "KZOS": "ОргСинт ао",
    "KZOSP": "ОргСинт ап",
    "PIKK": "ПИК ао",
    "PLZL": "Полюс",
    "PRMD": "ПРОМОМЕД",
    "RASP": "Распадская",
    "RENI": "Ренессанс",
    "ROSN": "Роснефть",
    "FEES": "Россети",
    "MRKU": "Россети Ур",
    "MRKC": "РоссЦентр",
    "RTKM": "Ростел-ао",
    "RTKMP": "Ростел-ап",
    "MRKV": "РсетВол ао",
    "LSNGP": "РСетиЛЭ-п",
    "MSRS": "РСетиМР ао",
    "MRKP": "РСетиЦП ао",
    "RUAL": "РУСАЛ ао",
    "HYDR": "РусГидро",
    "RNFT": "РуссНфт ао",
    "SMLT": "Самолет ао",
    "SBER": "Сбербанк",
    "SBERP": "Сбербанк-п",
    "CHMF": "СевСт-ао",
    "SGZH": "Сегежа",
    "SELG": "Селигдар",
    "AFKS": "Система ао",
    "SVCB": "Совкомбанк",
    "FLOT": "Совкомфлот",
    "SVAV": "СОЛЛЕРС",
    "SPBE": "СПБ Биржа",
    "SNGS": "Сургнфгз",
    "SNGSP": "Сургнфгз-п",
    "T": "Т-Техно ао",
    "TATN": "Татнфт 3ао",
    "TATNP": "Татнфт 3ап",
    "TGKA": "ТГК-1",
    "TGKB": "ТГК-2",
    "TRMK": "ТМК ао",
    "TRNFP": "Транснф ап",
    "PHOR": "ФосАгро ао",
    "HEAD": "Хэдхантер",
    "HNFG": "ХЭНДЕРСОН",
    "ELFV": "ЭЛ5Энер ао",
    "SFIN": "ЭН+ГРУП ао",
    "ESFA": "ЭсЭфАй ао",
    "UGLD": "ЮГК",
    "UPRO": "Юнипро ао",
    "YDEX": "ЯНДЕКС"
    }
    
    # Создаем папки для отчетов
    if not os.path.exists('Data_storage/reports'):
        os.makedirs('Data_storage/reports')
    if not os.path.exists('Data_storage/reports/quarterly'):
        os.makedirs('Data_storage/reports/quarterly')
    if not os.path.exists('Data_storage/reports/yearly'):
        os.makedirs('Data_storage/reports/yearly')

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }

    # Скачиваем квартальные отчеты
    print("Скачиваем квартальные отчеты...")
    for ticker, name in companies.items():
        url = f"https://smart-lab.ru/q/{ticker}/f/q/MSFO/download/"
        filename = f"Data_storage/reports/quarterly/{ticker}_quarterly.csv"
        
        try:
            print(f"Скачиваем квартальный отчет {name} ({ticker})...")
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"Успешно сохранено в {filename}")
        except Exception as e:
            print(f"Ошибка при скачивании {ticker}: {str(e)}")
        
        sleep(2)  # Пауза между запросами

    # Скачиваем годовые отчеты
    print("\nСкачиваем годовые отчеты...")
    for ticker, name in companies.items():
        url = f"https://smart-lab.ru/q/{ticker}/f/y/MSFO/download/"
        filename = f"Data_storage/reports/yearly/{ticker}_yearly.csv"
        
        try:
            print(f"Скачиваем годовой отчет {name} ({ticker})...")
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"Успешно сохранено в {filename}")
        except Exception as e:
            print(f"Ошибка при скачивании {ticker}: {str(e)}")
        
        sleep(2)  # Пауза между запросами

    print("\nЗагрузка завершена!")

if __name__ == "__main__":
    download_reports()

In [ ]:
import requests
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
import pandas as pd
import csv
import sys


def fetch_data(from_date, to_date):
    headers = {
        'Content-Type': 'text/xml; charset=utf-8',
        'SOAPAction': 'http://web.cbr.ru/KeyRateXML',
    }

    body = f'''<?xml version="1.0" encoding="utf-8"?> 
    <soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/"> 
        <soap:Body> 
            <KeyRateXML xmlns="http://web.cbr.ru/"> 
                <fromDate>{from_date}</fromDate> 
                <ToDate>{to_date}</ToDate> 
            </KeyRateXML> 
        </soap:Body> 
    </soap:Envelope> 
    '''

    response = requests.post('http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx',
                             headers=headers, data=body)

    if response.status_code == 200:
        return ET.fromstring(response.content)
    else:
        response.raise_for_status()


def parse_response(root):
    return [
        {
            'date': item.find('DT').text,
            'rate': float(item.find('Rate').text.replace(',', '.'))
        }
        for item in root.iter('KR')
    ]


def create_dataframe(data):
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date']).dt.date
    return df


def fill_missing_dates(df, start_date, end_date):
    all_dates = pd.date_range(start=start_date, end=end_date).date
    df = df.set_index('date').reindex(all_dates, method='ffill').reset_index()
    df.columns = ['date', 'rate']
    return df


def save_to_csv(df):
    df.to_csv(sys.stdout, index=False, encoding='UTF-8', quoting=csv.QUOTE_NONNUMERIC, sep=";")


def main():
    # Получаем сегодняшнюю дату и дату 10 лет назад
    today = datetime.now()
    ten_years_ago = today - timedelta(days=365 * 10)  # приблизительно 10 лет назад

    # Форматируем даты в формате YYYY-MM-DD
    from_date = ten_years_ago.strftime('%Y-%m-%d')
    to_date = today.strftime('%Y-%m-%d')

    # Запрашиваем данные через SOAP API
    root = fetch_data(from_date, to_date)
    data = parse_response(root)
    df = create_dataframe(data)

    # Заполняем пропущенные даты (если данные не приходят за каждый день)
    df = fill_missing_dates(df, ten_years_ago.date(), today.date())

    # Сортируем по дате и выводим CSV в стандартный вывод
    df = df.sort_values(by='date')
    df.to_csv("../data/external/macro/key_rate.csv", index=False, encoding='UTF-8', quoting=csv.QUOTE_NONNUMERIC, sep=";")


if __name__ == "__main__":
    main()